# **Tarea 2**

## _Punto 1:_

### 1. Modelado del problema como un MDP

Modele este problema como un MDP. Detalle todos los elementos del MDP:

- **Estados**  
- **Recompensas**  
- **Acciones**  
- **Dinámica de transición**:  
  $$ p(s', r \mid s, a) \quad \forall s, s', r, a $$  
- **Factor de descuento**:  
  $$ \gamma $$

In [1]:
import numpy as np

In [9]:
# Definir el número de estados y acciones
estados = 100  # Casillas del tablero
acciones = [-1, 1]  # -1: Retroceder, 1: Avanzar

In [10]:
probabilidades = [0.1, 0.15, 0.2, 0.2, 0.15, 0.1]  # Probabilidades del dado cargado

In [11]:

# Definir posiciones de escaleras y serpientes
escaleras = {4: 14, 9: 31, 21: 42, 28: 84, 36: 44, 51: 67, 71: 91, 80: 99}
serpientes = {16: 6, 48: 26, 49: 11, 56: 53, 62: 19, 64: 60, 87: 24, 93: 73, 95: 75, 98: 78}

In [12]:

# Casillas de victoria y derrota
estado_ganar = [80, 100]  # Casillas azules
estado_perder = [16, 48, 49, 56, 62, 64, 87, 93, 95, 98]  # Casillas rojas

In [13]:
def probTransicion(s, a, probabilidades):
    """ Calcula la distribución de probabilidad para el próximo estado."""
    prob_nuevoEstado = np.zeros(estados + 1)  # De 1 a 100
    
    for lanzamiento, prob in enumerate(probabilidades, start=1):
        nuevoEstado = s + a * lanzamiento
        
        # Rebote en los extremos
        if nuevoEstado > 100:
            nuevoEstado = 100 - (nuevoEstado - 100)
        elif nuevoEstado < 1:
            nuevoEstado = 1 + (1 - nuevoEstado)
        
        # Escaleras y serpientes
        if nuevoEstado in escaleras:
            nuevoEstado = escaleras[nuevoEstado]
        elif nuevoEstado in serpientes:
            nuevoEstado = serpientes[nuevoEstado]
        
        prob_nuevoEstado[nuevoEstado] += prob
    
    return prob_nuevoEstado

In [14]:
def funRecompensa(s):
    """ Define la función de recompensa."""
    if s in estado_ganar:
        return 1  # Victoria
    elif s in estado_perder:
        return -1  # Derrota
    return -0.01  # Penalización por movimiento


In [15]:
# Factor de descuento
gamma = 0.9